In [1]:
import cvxpy as cp


In [2]:
cp.installed_solvers()

['ECOS', 'ECOS_BB', 'SCS', 'SUPERSCS', 'MOSEK', 'LS']

In [25]:
# SuperSCS LASSO Benchmark

import numpy as np
import cvxpy as cp
import scipy as sp

import scipy.sparse as sps

np.random.seed(0)
# Variable declarations
n = 10000
m = 2000

s = int(np.ceil(n/10))
x_true = np.vstack((np.random.randn(s, 1), np.zeros((n-s, 1))))
x_true = np.random.permutation(x_true)

density = 0.1
rcA = 0.1
A = sps.random(m, n, density, data_rvs = np.random.randn)

b = A*x_true + 0.1*np.random.randn(m, 1)
mu = 1


# Problem construction
x = cp.Variable(n)

prob = cp.Problem(cp.Minimize(0.5*cp.sum_squares(A*x) + mu*cp.norm1(x)))

problems = {"superscs_lasso" : prob}



In [109]:
problemID = "covsel_0"

import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

np.random.seed(0)

m = 100
n = 200
lam = float(0.1)

import scipy.sparse as sps

A = sps.rand(n,n, 0.01)
A = np.asarray(A.T.dot(A).todense() + 0.1*np.eye(n))
L = np.linalg.cholesky(np.linalg.inv(A))
X = np.random.randn(m,n).dot(L.T)
S = X.T.dot(X)/m
W = np.ones((n,n)) - np.eye(n)

Theta = cp.Variable(n,n)


# Problem construction
prob = None
opt_val = None

prob = cp.Problem(cp.Minimize(
        lam*cp.norm1(cp.mul_elemwise(W,Theta)) +
        cp.sum_entries(cp.mul_elemwise(S,Theta)) -
        cp.log_det(Theta)))


# For debugging individual problems:
if __name__ == "__main__":
    prob.solve(solver="SUPERSCS")
    print("status:", prob.status)
    print("optimal value:", prob.value)
    print("true optimal value:", opt_val)

SolverError: Solver does not support positive semidefinite cone.

In [71]:
# Dumb Least Squares SCS > SUPERSCS
n = 80
m = 100
np.random.seed(1)

x = cp.Variable(n)
x_0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A.dot(x_0) + 0.5*np.random.randn(m)


# Problem construction
prob = None
opt_val = None

objective = cp.Minimize(cp.sum_squares(A*x - b))
prob = cp.Problem(objective)

In [74]:
# Basis Pursuit SCS > SUPERSCS

import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

import scipy.sparse as sps
n = 300
m = 100
np.random.seed(0)

x = cp.Variable(n)
A = np.random.rand(m, n)
x0 = sps.rand(n, 1, 0.1)

b = A*x0


# Problem construction
prob = None
opt_val = None

prob = cp.Problem(cp.Minimize(cp.norm1(x)), [A*x == b])


In [85]:
# lp_1 SUPERSCS > SCS
m = 30
n = 300
np.random.seed(1)

A = np.random.rand(m, n)

y = (np.random.rand(n) - 0.5)
x_hat = -np.select([y < 0], [y])
Lambda = np.select([y >= 0], [y]) # x_hat^T Lambda = 0

mu = np.random.rand(m)
c = Lambda - A.T.dot(mu)
b = A.dot(x_hat)
x = cp.Variable(n)


# Problem construction
prob = None
opt_val = None

objective = cp.Minimize(c*x)
constraints = [A*x == b, x >= 0]

prob = cp.Problem(objective, constraints)

In [96]:
# chebyshev_0 SUPERSCS > SCS
import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

def normalized_data_matrix(m, n, mu):
    if mu == 1:
        # dense
        A = np.random.randn(m, n)
        A /= np.sqrt(np.sum(A**2, 0))
    else:
        # sparse
        A = sp.rand(m, n, mu)
        A.data = np.random.randn(A.nnz)
        N = A.copy()
        N.data = N.data**2
        A = A*sp.diags([1 / np.sqrt(np.ravel(N.sum(axis=0)))], [0])

    return A

np.random.seed(0)
m = 100
n = 200
k = 10
A = [normalized_data_matrix(m,n,1) for i in range(k)]
B = normalized_data_matrix(k,n,1)
c = np.random.rand(k)

x = cp.Variable(n)
t = cp.Variable(k)


# Problem construction
prob = None
opt_val = None


f = cp.max_entries(t+cp.abs(B*x-c))
C = []
for i in range(k):
    C.append(cp.pnorm(A[i]*x, 2) <= t[i])

prob = cp.Problem(cp.Minimize(f), C)

In [101]:
# least_abs_dev_0 SUPERSCS > SCS

import cvxpy as cp
import numpy as np
import scipy as sp

# Variable declarations

import scipy.sparse as sps

np.random.seed(0)
m = 5000
n = 200

A = np.random.randn(m,n);
A = A*sps.diags([1 / np.sqrt(np.sum(A**2, 0))], [0])
b = A.dot(10*np.random.randn(n))

k = max(m//50, 1)
idx = np.random.randint(0, m, k)
b[idx] += 100*np.random.randn(k)

x = cp.Variable(n)


# Problem construction
prob = None
opt_val = None

prob = cp.Problem(cp.Minimize(cp.norm1(A*x - b)))

In [108]:
results = prob.solve(solver = "SUPERSCS", verbose = True, eps = 1e-5)
print("solve time", prob.solver_stats.solve_time)
print("status", prob.status)
print("opt_val", prob.value)

solve time 4.381547678
status optimal
opt_val 7094.74672291801


In [106]:
prob.solve(solver = "SCS", verbose = True, eps = 1e-5)
print("solve time", prob.solver_stats.solve_time)
print("status", prob.status)
print("opt_val", prob.value)

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2010000, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 5200, constraints m = 10000
Cones:	linear vars: 10000
Setup time: 3.31e-01s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  1.53e-01 
    60| 1.01e-06  1.10e-06  7.52e-08  7.09e+03  7.09e+03  4.17e-12  1.70e+00 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 1.70e+00s
	Lin-sys:

In [107]:
prob.solve(solver = "MOSEK", verbose = True)
print("solve time", prob.solver_stats.solve_time)
print("status", prob.status)
print("opt_val", prob.value)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 10000           
  Cones                  : 0               
  Scalar variables       : 5200            
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Interior-point optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.01            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.80    
Optimizer  - threads              

In [15]:
import inspect

In [17]:
superscs.solve.__code__.co_argcount

2

In [124]:
for constr in prob.canonicalize()[1]:
    print(isinstance(constr, cp.constraints.exponential.ExpCone))


False
False
False
False
False
False
False
False
False
True


In [134]:
X = cp.Semidef(10)
A = np.eye(10)
Y = cp.Int(3)
b = np.array([4,5,6])
Z = cp.Bool(2)
c = np.array([0, 1])

In [143]:
testprob = cp.Problem(cp.Minimize(0), [X >> A, Y == b,  Z == c])

In [144]:
testprob.canonicalize()

(LinOp(type='scalar_const', size=(1, 1), args=[], data=0),
  LinEqConstr(expr=LinOp(type='sum', size=(3, 1), args=[LinOp(type='variable', size=(3, 1), args=[], data=288), LinOp(type='dense_const', size=(3, 1), args=[], data=matrix([[-4.],
        [-5.],
        [-6.]]))], data=None), constr_id=308, size=(3, 1)),
  LinEqConstr(expr=LinOp(type='sum', size=(2, 1), args=[LinOp(type='variable', size=(2, 1), args=[], data=289), LinOp(type='dense_const', size=(2, 1), args=[], data=matrix([[-0.],
        [-1.]]))], data=None), constr_id=310, size=(2, 1))])

SolverError: The solver SCS cannot solve the problem because it cannot solve mixed-integer problems.

In [146]:
from cvxpy.constraints.soc_axis import SOC_Axis
from cvxpy.constraints.second_order import SOC

In [153]:
issubclass(SOC_Axis, SOC)

True